# Using NCO utilities ncks and ncrcat to subset CETB files by location and concatenate by time series

To begin, you need to know the map coordinates of the spatial subset you want.

You can use various utilities to do this, we have written the python package cetbtools.ease2conv to help with it.

For this example, I want the upper left quadrant of the EASE2_N25km grid, which is bounded by
the upper left corner of the grid and the North Pole at the center of the grid.

In [ ]:
%pylab notebook
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from cetbtools.ease2conv import Ease2Transform

In [ ]:
subULrow, subULcol = -0.5, -0.5
subLRrow, subLRcol = 359.5, 359.5
N25 = Ease2Transform("EASE2_N25km")
subULx, subULy = N25.grid_to_map(subULrow, subULcol)
subLRx, subLRy = N25.grid_to_map(subLRrow, subLRcol)
print "Subset UL x,y = %.3f, %.3f" % (subULx, subULy)
print "Subset LR x,y = %.3f, %.3f" % (subLRx, subLRy)

These are coordinates for the CRREL Baltic subset we are interested in:
<pre>
     col  row
UL:  418  478
UR:  428  478
LR:  428  506
LL:  418  506
</pre>


In [ ]:
subULrow, subULcol = 477.5, 417.5
subLRrow, subLRcol = 506.5, 428.5
N25 = Ease2Transform("EASE2_N25km")
subULx, subULy = N25.grid_to_map(subULrow, subULcol)
subLRx, subLRy = N25.grid_to_map(subLRrow, subLRcol)
print "Subset UL x,y = %.3f, %.3f" % (subULx, subULy)
print "Subset LR x,y = %.3f, %.3f" % (subLRx, subLRy)

So these min/max values for x and y can be passed to ncks to subset the TB variable from a CETB file with:

ncks -d x,-9000000.,0. -d y,0.,9000000. -v TB NSIDC-0630-EASE2_N25km-AQUA_AMSRE-2011270-18H-E-GRD-RSS-v1.0.nc d270.18H.ul.nc

ncks -d x,1450000.,1725000. -d y,-3675000.0,-2950000.0 -v TB NSIDC-0630-EASE2_N25km-AQUA_AMSRE-2011270-18H-E-GRD-RSS-v1.0.nc Baltic.EASE2_N25km-AQUA_AMSRE-2011270-18H-E-GRD.nc

This subsets the upper left quadrant of the EASE2_N25km grid into the file d001.19H.ul.nc.

In [ ]:
def make_blank_file(src, dst, new_time):
    # copy the src file to the dst filename
    call(['cp', src, dst])
    
    # change the time value in the new file to new_time
    fid = Dataset( dst, "a", format="NETCDF4")
    
    fid.variables['time'][:] = new_time
    
    # fill the TB array with zeroes
    data = fid.variables['TB'][:]
    fid.variables['TB'][:] = np.zeros_like(data)
    
    fid.close()


In [ ]:
%cd /projects/PMESDR/vagrant/AMSRE_1836/2003
%ls

In [ ]:
import glob
list = sort(glob.glob("*18H-M-GRD*"))
list = list[:3]
list

In [ ]:
from subprocess import call
import shlex

In [ ]:
for file in list:
    outfile = "/projects/PMESDR/vagrant/brodzik/ncks_tests/Baltic.%s" % file
    unlfile = "/projects/PMESDR/vagrant/brodzik/ncks_tests/Baltic.unlimited.%s" % file
    print("Next in : %s" % file)
    print("Next out: %s" % outfile)
    print("next unl out: %s" % unlfile)
    call(['ncks', '-d', 'x,1450000.,1725000.', '-d', 'y,-3675000.0,-2950000.0',
          '-v', 'TB', file, outfile])
    call(['ncks', '-O', '--mk_rec_dmn', 'time', outfile, unlfile])
    print("Done")

In [ ]:
#%ls -las /projects/PMESDR/vagrant/brodzik/ncks_tests
%pwd
%ls

In [ ]:
%cd /projects/PMESDR/vagrant/brodzik/ncks_tests
#Then for a full list of files you can concatenate them in the time dimension with:
call(['ncrcat', '-O', 'Baltic.unlimited\*', 'test.all_days.nc'])

In [ ]:
fid.close()

In [ ]:
filename = "test.nc"
fid = Dataset( filename, "r", format="NETCDF4")
fid

In [ ]:
tb = fid.variables['TB'][:]

In [ ]:
fid.variables['time'][:]

In [ ]:
np.shape(tb)

In [ ]:
tb = np.squeeze(tb)
np.shape(tb)

In [ ]:
fig, ax = plt.subplots(1,3)
for i in np.arange(3):
    ax[i].imshow(tb[i,:,:])

Then for each subset file you must change "time" from a fixed dimension to a record (unlimited dimension), using

ncks -O --mk_rec_dmn time d002.19H.ul.nc d002.19H.ul.new.nc

(this will change the ncdump -h information from: 

<blockquote>
<p>dimensions:
	<p>time = 1 ;
</blockquote>
    
to   

<blockquote>
<p>dimensions:
	<p>time = UNLIIMITED ; // (1 currently)
</blockquote>
Then for a full list of files you can concatenate them in the time dimension with:

ncrcat -O *.new.nc all_days.19H.ul.nc

Thanks to this NASA recipe %pylab inline
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as npfor tips here:

http://disc.sci.gsfc.nasa.gov/recipes/?q=recipes/How-to-Concatenate-the-Time-Dimension-of-netCDF-Files-with-NCO

In [ ]:
combfile = "d001-003.19H.ul.nc"
combfid = Dataset(combfile, 'r', format="NETCDF")
combfid

In [ ]:
np.shape(combfid.variables["TB"][:])

In [ ]:
slice0 = combfid.variables["TB"][:][0,:,:]

In [ ]:
np.shape(slice)

In [ ]:
plt.imshow(slice0)

In [ ]:
slice1 = combfid.variables["TB"][:][1,:,:]
plt.imshow(slice1)

In [ ]:
slice2 = combfid.variables["TB"][:][2,:,:]
plt.imshow(slice2)

# Test pixel map coordinates for Melt Onset analysis

For Karakoram pixel at 31.1N, 75.8E:

In [ ]:
lat, lon = 31.1, 75.8
row, col = N25.geographic_to_grid(lat, lon)
x, y = N25.grid_to_map(row, col)
print row,col
print x, y

For Himalaya pixel at 38.1N, 88.3E:

In [ ]:
lat, lon = 38.1, 88.3
row, col = N25.geographic_to_grid(lat, lon)
x, y = N25.grid_to_map(row, col)
print row,col
print x, y

In [ ]:
src = "Baltic.unlimited.NSIDC-0630-EASE2_N25km-AQUA_AMSRE-2003001-18H-M-GRD-RSS-v1.0.nc"
dst = "test.copy.nc"
new_time = 11323 + 3
make_blank_file(src, dst, new_time)

